# Inference for OpenSora

Define global variables. You should change the following variables according to your setting.

In [ ]:
# global variables
ROOT = ".."
cfg_path = f"{ROOT}/configs/opensora-v1-2/inference/sample.py"
ckpt_path = "/home/lishenggui/projects/sora/Open-Sora-dev/outputs/207-STDiT3-XL-2/epoch0-global_step9000/"
vae_path = f"{ROOT}/pretrained_models/vae-pipeline"
save_dir = f"{ROOT}/samples/samples_notebook/"
device = "cuda:0"

Import necessary libraries and load the models.

In [ ]:
import os
from pprint import pformat

import colossalai
import torch
import torch.distributed as dist
from colossalai.cluster import DistCoordinator
from mmengine.runner import set_random_seed
from tqdm.notebook import tqdm

from opensora.acceleration.parallel_states import set_sequence_parallel_group
from opensora.datasets import save_sample, is_img
from opensora.datasets.aspect import get_image_size, get_num_frames
from opensora.models.text_encoder.t5 import text_preprocessing
from opensora.registry import MODELS, SCHEDULERS, build_module
from opensora.utils.config_utils import read_config
from opensora.utils.inference_utils import (
    append_generated,
    apply_mask_strategy,
    collect_references_batch,
    extract_json_from_prompts,
    extract_prompts_loop,
    get_save_path_name,
    load_prompts,
    prepare_multi_resolution_info,
)
from opensora.utils.misc import all_exists, create_logger, is_distributed, is_main_process, to_torch_dtype

In [ ]:
torch.set_grad_enabled(False)

# == parse configs ==
cfg = read_config(cfg_path)
cfg.model.from_pretrained = ckpt_path
cfg.vae.from_pretrained = vae_path

# == device and dtype ==
cfg_dtype = cfg.get("dtype", "fp32")
assert cfg_dtype in ["fp16", "bf16", "fp32"], f"Unknown mixed precision {cfg_dtype}"
dtype = to_torch_dtype(cfg.get("dtype", "bf16"))
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

set_random_seed(seed=cfg.get("seed", 1024))

In [ ]:
# == build text-encoder and vae ==
text_encoder = build_module(cfg.text_encoder, MODELS, device=device)
vae = build_module(cfg.vae, MODELS).to(device, dtype).eval()

# == build diffusion model ==
input_size = (None, None, None)
latent_size = vae.get_latent_size(input_size)
model = (
    build_module(
        cfg.model,
        MODELS,
        input_size=latent_size,
        in_channels=vae.out_channels,
        caption_channels=text_encoder.output_dim,
        model_max_length=text_encoder.model_max_length,
    )
    .to(device, dtype)
    .eval()
)
text_encoder.y_embedder = model.y_embedder  # HACK: for classifier-free guidance

Define inference function.

In [ ]:
start_idx = 0
multi_resolution = cfg.get("multi_resolution", None)
batch_size = cfg.get("batch_size", 1)


def inference(
    prompts=cfg.get("prompt", None),
    image_size=None,
    num_frames=None,
    resolution=None,
    aspect_ratio=None,
    mask_strategy=None,
    reference_path=None,
    num_sampling_steps=None,
    cfg_scale=None,
    seed=None,
    fps=cfg.fps,
    num_sample=cfg.get("num_sample", 1),
    loop=cfg.get("loop", 1),
    condition_frame_length=cfg.get("condition_frame_length", 5),
    align=cfg.get("align", None),
    sample_name=cfg.get("sample_name", None),
    prompt_as_path=cfg.get("prompt_as_path", False),
    disable_progress=False,
):
    global start_idx
    os.makedirs(save_dir, exist_ok=True)
    if seed is not None:
        set_random_seed(seed=seed)
    if not isinstance(prompts, list):
        prompts = [prompts]
    if mask_strategy is None:
        mask_strategy = [""] * len(prompts)
    if reference_path is None:
        reference_path = [""] * len(prompts)
    save_fps = cfg.fps // cfg.get("frame_interval", 1)
    if num_sampling_steps is not None:
        cfg.scheduler["num_sampling_steps"] = num_sampling_steps
    if cfg_scale is not None:
        cfg.scheduler["scale"] = cfg_scale
    scheduler = build_module(cfg.scheduler, SCHEDULERS)
    ret_path = []

    # == prepare video size ==
    if image_size is None:
        assert (
            resolution is not None and aspect_ratio is not None
        ), "resolution and aspect_ratio must be provided if image_size is not provided"
        image_size = get_image_size(resolution, aspect_ratio)
    num_frames = get_num_frames(num_frames)
    input_size = (num_frames, *image_size)
    latent_size = vae.get_latent_size(input_size)

    # == Iter over all samples ==
    for i in tqdm(range(0, len(prompts), batch_size), disable=disable_progress):
        # == prepare batch prompts ==
        batch_prompts = prompts[i : i + batch_size]
        ms = mask_strategy[i : i + batch_size]
        refs = reference_path[i : i + batch_size]

        batch_prompts, refs, ms = extract_json_from_prompts(batch_prompts, refs, ms)
        refs = collect_references_batch(refs, vae, image_size)

        # == multi-resolution info ==
        model_args = prepare_multi_resolution_info(
            multi_resolution, len(batch_prompts), image_size, num_frames, fps, device, dtype
        )

        # == Iter over number of sampling for one prompt ==
        for k in range(num_sample):
            # == prepare save paths ==
            save_paths = [
                get_save_path_name(
                    save_dir,
                    sample_name=sample_name,
                    sample_idx=start_idx + idx,
                    prompt=batch_prompts[idx],
                    prompt_as_path=prompt_as_path,
                    num_sample=num_sample,
                    k=k,
                )
                for idx in range(len(batch_prompts))
            ]

            # NOTE: Skip if the sample already exists
            # This is useful for resuming sampling VBench
            if prompt_as_path and all_exists(save_paths):
                continue

            # == Iter over loop generation ==
            video_clips = []
            for loop_i in range(loop):
                batch_prompts_loop = extract_prompts_loop(batch_prompts, loop_i)
                batch_prompts_cleaned = [text_preprocessing(prompt) for prompt in batch_prompts_loop]

                # == loop ==
                if loop_i > 0:
                    refs, ms = append_generated(vae, video_clips[-1], refs, ms, loop_i, condition_frame_length)

                # == sampling ==
                z = torch.randn(len(batch_prompts), vae.out_channels, *latent_size, device=device, dtype=dtype)
                masks = apply_mask_strategy(z, refs, ms, loop_i, align=align)
                samples = scheduler.sample(
                    model,
                    text_encoder,
                    z=z,
                    prompts=batch_prompts_cleaned,
                    device=device,
                    additional_args=model_args,
                    progress=False,
                    mask=masks,
                )
                samples = vae.decode(samples.to(dtype), num_frames=num_frames)
                video_clips.append(samples)

            # == save samples ==
            if is_main_process():
                for idx, batch_prompt in enumerate(batch_prompts):
                    save_path = save_paths[idx]
                    video = [video_clips[i][idx] for i in range(loop)]
                    for i in range(1, loop):
                        video[i] = video[i][:, condition_frame_length:]
                    video = torch.cat(video, dim=1)
                    path = save_sample(
                        video,
                        fps=save_fps,
                        save_path=save_path,
                        verbose=False,
                    )
                    ret_path.append(path)
        start_idx += len(batch_prompts)
    return ret_path

In [ ]:
from IPython.display import Video, Image, display


def display_results(paths):
    for path in paths:
        if is_img(path):
            display(Image(path))
        else:
            display(Video(path, embed=True))


def reset_start_idx():
    global start_idx
    start_idx = 0


ALL_ASPECT_RATIO = ["1:1", "16:9", "9:16", "3:4", "4:3", "1:2", "2:1"]


def inference_all_aspects(prompts, resolution, num_frames, *args, **kwargs):
    paths = []
    for aspect_ratio in tqdm(ALL_ASPECT_RATIO):
        paths.extend(
            inference(
                prompts,
                resolution=resolution,
                num_frames=num_frames,
                aspect_ratio=aspect_ratio,
                disable_progress=True,
                *args,
                **kwargs
            )
        )
    return paths

## Inference for OpenSora

Sample code for inference for OpenSora.

In [ ]:
paths = inference(
    ["a man.", "a woman"],
    resolution="240p",
    aspect_ratio="1:1",
    num_frames="1x",
    num_sampling_steps=30,
    cfg_scale=7.0,
)
display_results(paths)

Sample all aspect ratios.

In [ ]:
PROMPT = "a boy."
paths = inference_all_aspects(
    PROMPT,
    resolution="240p",
    num_frames="1x",
    num_sampling_steps=30,
    cfg_scale=7.0,
)
display_results(paths)

Sample all resolution and length.

In [ ]:
PROMPT = "a boy."
sample_cfg = {
    "144p": [1, "1x", "2x", "4x", "8x"],
    "240p": [1, "1x", "2x", "4x", "8x"],
    "360p": [1, "1x", "2x", "4x"],
    "480p": [1, "1x", "2x", "4x"],
    "720p": [1, "1x", "2x"],
}
all_paths = []
for resolution, num_frames in sample_cfg.items():
    for num_frame in num_frames:
        print(f"Resolution: {resolution}, Num Frames: {num_frame}")
        paths = inference(
            PROMPT,
            resolution=resolution,
            num_frames=num_frame,
            aspect_ratio="9:16",
            num_sampling_steps=30,
            cfg_scale=7.0,
            disable_progress=True,
        )
        display_results(paths)
        all_paths.extend(paths)

Sample all resolution, length, and aspect ratios.

In [ ]:
PROMPT = "a boy."
sample_cfg = {
    "144p": [1, "1x", "2x", "4x", "8x"],
    "240p": [1, "1x", "2x", "4x", "8x"],
    "360p": [1, "1x", "2x", "4x"],
    "480p": [1, "1x", "2x", "4x"],
    "720p": [1, "1x", "2x"],
}
all_paths = []
for resolution, num_frames in sample_cfg.items():
    for num_frame in num_frames:
        paths = inference_all_aspects(
            PROMPT,
            resolution=resolution,
            num_frames=num_frames,
            num_sampling_steps=30,
            cfg_scale=7.0,
        )
        display_results(paths)
        all_paths.extend(paths)